In [1]:
#import course;course.header()

# Find 10 closest cities to a given latidue and longitude point on earth

Use any data structure that you can get your hands on and load the cities data. Write a function that creates the data structure withthe following function signature

In [2]:
def create_data_stucture(map_data_file):
    """Create a data structure from a data_file that allows rapid quering positions on the map

    Args:
        map_data_file (Path) csv file with the following columns:
            city,lat,lng,country,iso3,local_name,population,continent

    Returns
        data_structure that can be used with find_closest_cities

    HINT:
        BSP buildup using recursive functions
    """

In [3]:
# Daten einlesen, Dictionary, City zu Tupel aus Koordinaten

cities_dict = {}

with open("..\..\data\cities.csv", "r", encoding="utf8") as data:
    next(data)
    for line in data:
        if line[0] == '"':
            splitted_line = line.split(",")
            cities_dict[splitted_line[0]] = (splitted_line[2], splitted_line[3])
        else:
            splitted_line = line.split(",")
            cities_dict[splitted_line[0]] = (splitted_line[1], splitted_line[2]) 


In [4]:
#print(cities_dict)

In [5]:
# Methode um Distanz zwischen Punkt und Stadt zu berechnen (ohne Erdkrümmung)
import numpy as np

def calculate_distance(city, point):
    lat_city = cities_data[city][0]
    lng_city = cities_data[city][1]
    difference_latidue = abs(lat_city - point[0])
    difference_longitude = abs(lng_city - point[1])
    if difference_longitude > 180:
        difference_longitude = 360 - difference_longitude
    distance = np.sqrt(difference_longitude**2 + difference_latidue**2)


In [7]:
# Liste, welche die ersten 10 Städte speichert, eine Neue hinzufügt, und dann die 11. Stadt löscht

point = (0.000, 0.000)
print(geopy.distance.distance(cities_dict["Tokyo"], point))

distances_dict = {}

for key in cities_dict:
    distances_dict[key] = geopy.distance.distance(cities_dict[key], point)


14274.268873103454 km


In [8]:
distance_dict_sorted = {key: value for key, value in sorted(distances_dict.items(), key= lambda item: item[1])}

import itertools
nearest_dict = dict(itertools.islice(distance_dict_sorted.items(), 10))
nearest_list = list(nearest_dict.keys())

print(nearest_list)

['Sekondi', 'Cape Coast', 'Winneba', 'Accra', 'Tema', 'Palé', 'Koforidua', 'Aflao', 'Lomé', 'Aboisso']


Write a query function with the following signature

In [9]:
def find_closest_cities(point, data_structure):
    """Find ten close cities to the given point on the world given the data structure
    
    Args:
        point (tuple of floats) latitude and longitude
    
        data_structure (you choose) to be queries using the point

    Returns:
        list of 10 cities that are closest to the point 
    """
    return

Creating a kd-tree with the others:

In [1]:
import csv
map_data = []
with open('..\..\data\cities.csv', encoding="utf-8") as cities_file:
    map_data_reader = csv.DictReader(cities_file, delimiter=',')
    for line in map_data_reader:
        map_data.append([line['city'], [float(line['lat']), float(line['lng'])]])

map_data[14990:15000]

[['Yayladağı', [35.9025, 36.0603]],
 ['Nanzhiqiu', [37.7492, 115.2357]],
 ['Lanciano', [42.2312, 14.3905]],
 ['Cutral-Có', [-38.9333, -69.2333]],
 ['Matinhos', [-25.8178, -48.5428]],
 ['Lage', [51.9833, 8.8]],
 ['Vichuga', [57.2, 41.9167]],
 ['Santana do Paraíso', [-19.3639, -42.5689]],
 ['Shenjiabang', [30.5783, 120.8193]],
 ['Gahanna', [40.0251, -82.8637]]]

In [2]:
# child 1 -> smaller number, child 2 -> bigger number
# orientation = 0 --> lat, orientation = 1 ---> lng
import numpy as np
def create_data_stucture(cities, orientation=0):
    
    child1 = []
    child2 = []
    median_cities= []

    if len(cities) % 2 == 0:
        median = np.median([cities[city][1][orientation] for city in range(len(cities)-1)])
    elif len(cities) % 2 == 1:
        try:
            median = np.median([cities[city][1][orientation] for city in range(len(cities))])
        except:
            print('AAAA {}'.format(cities))
    else:
        print("You idiot!")

    for city_data in cities:
        if city_data[1][orientation] < median:
            child1.append(city_data)
        elif city_data[1][orientation] > median:
            child2.append(city_data)
        elif city_data[1][orientation] == median:
            median_cities.append(city_data)
        else:
            print("No median city??")

    median_city = median_cities[0]

    if len(median_cities) >= 2:
        if len(child1)>len(child2):
            for median_to_child in median_cities[1:]:
                child2.append(median_to_child)
        elif len(child1) <= len(child2):
            for median_to_child in median_cities[1:]:
                child1.append(median_to_child)

    if orientation == 0:
        orientation = 1
    elif orientation == 1:
        orientation = 0
    else:
        print("How is it even possible?")

    if child1 == []:
        if child2 == []:
            return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': None,
            'child2': None,
            'leaf': True}
        elif len(child2)==1:
            return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': None,
            'child2': {'city':[child2[0][0], child2[0][1][0], child2[0][1][1]],
                        'child1':None,
                        'child2':None,
                        'leaf':True},
            'leaf': False}
        elif len(child2) >= 2:
            return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': None,
            'child2':create_data_stucture(child2, orientation=orientation),
            'leaf': False}
    elif len(child1) == 1:
        if child2 == []:
            return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': {'city':[child1[0][0], child1[0][1][0], child1[0][1][1]],
                        'child1':None,
                        'child2':None,
                        'leaf':True},
            'child2': None,
            'leaf': False}
        elif len(child2) == 1:
            return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': {'city':[child1[0][0], child1[0][1][0], child1[0][1][1]],
                        'child1':None,
                        'child2':None,
                        'leaf':True},
            'child2': {'city':[child2[0][0], child2[0][1][0], child2[0][1][1]],
                        'child1':None,
                        'child2':None,
                        'leaf':True},
            'leaf': False}
        elif len(child2) >= 2:
            return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': {'city':[child1[0][0], child1[0][1][0], child1[0][1][1]],
                        'child1':None,
                        'child2':None,
                        'leaf':True},
            'child2': create_data_stucture(child2, orientation=orientation),
            'leaf': False}
    elif len(child1) >=2:
        if child2 == []:
            return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': create_data_stucture(child1, orientation=orientation),
            'child2': None,
            'leaf': False}
        elif len(child2) ==1 :
             return_tree={'city': [median_city[0], median_city[1][0], median_city[1][1]],
            'child1': create_data_stucture(child1, orientation=orientation),
            'child2': {'city':[child2[0][0], child2[0][1][0], child2[0][1][1]],
                        'child1':None,
                        'child2':None,
                        'leaf':True},
            'leaf': False}
        elif len(child2) >= 2:
            return_tree = {'city': [median_city[0], median_city[1][0], median_city[1][1]], 
                        'child1':create_data_stucture(child1, orientation=orientation), 
                        'child2':create_data_stucture(child2, orientation=orientation),
                        'leaf': False}
  
    return return_tree

In [3]:
map_tree = create_data_stucture(map_data)